In [79]:
import tweepy
import pandas as pd
import numpy as np
import re
import emoji
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [80]:
# you have to add twitter Developer account api keys
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''


In [ ]:
latitude = 53.43084848150311   # geographical centre of search
longitude = -2.9608694861754525    # geographical centre of search
max_range = 300      # search range in kilometres

In [81]:
#Authenticate with credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [82]:
key_words = ['Park -filter:retweets']
limit = 10000

In [83]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [84]:
def TextClean(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'@[a-z0-9_]\S+', '', tweet)
    tweet = re.sub(r'#[a-z0-9_]\S+', '', tweet)
    tweet = re.sub(r'&[a-z0-9_]\S+', '', tweet)
    tweet = re.sub(r'[?!.+,;$%&"]+', '', tweet)
    tweet = re.sub(r'rt[\s]+', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = re.sub(r'\$', '', tweet)
    tweet = re.sub(r'rt+', '', tweet)
    tweet = re.sub(r'https?:?\/\/\S+', '', tweet)
    
    return tweet

In [85]:
def tweet_search(key_word):
    i = 0
    tweets_df = pd.DataFrame(columns=['Datetime', 'Tweet', 'Username', 'Retweets', 'Followers'])
    for tweet in tweepy.Cursor(api.search_tweets, q = key_word,geocode = "%f,%f,%dkm" % (latitude, longitude, max_range) ,count = 100, lang = 'en', tweet_mode = 'extended').items():
        print('Tweets downloaded:', i, '/', limit, end = '\r')
        if tweet.user.followers_count > 200:
            tweets_df = tweets_df.append({'Datetime': tweet.created_at,
                                          'Tweet': tweet.full_text,
                                          'Username': tweet.user.screen_name,
                                          'Retweets': tweet.retweet_count,
                                          'Followers': tweet.user.followers_count}, ignore_index = True)
            i += 1
        if i >= limit:
            break
        else:
            pass
        
    tweets_df['Datetime'] = pd.to_datetime(tweets_df['Datetime'], format = '%Y.%m.%d %H:%M:%S')
    tweets_df.drop_duplicates(subset = ['Tweet'], inplace = True)
    tweets_df['CleanTweet'] = tweets_df['Tweet'].apply(TextClean)
    tweets_df['CleanTweet2'] = tweets_df['Tweet'].apply(TextClean)
    tweets_df['CleanTweet2'] = tweets_df['CleanTweet2'].apply(remove_emojis)
    tweet_tokenizer = TweetTokenizer()
    tweets_df['CleanTweet'] = tweets_df['CleanTweet'].apply(tweet_tokenizer.tokenize)
    tweets_df['CleanTweet'] = [', '.join(map(str, token)) for token in tweets_df['CleanTweet']]
    
    return tweets_df

In [86]:
tweets_df = pd.DataFrame()
for x in key_words:
    df =tweet_search(x)
    tweets_df=tweets_df.append(df, ignore_index=True)
tweets_df

,Datetime,Tweet,Username,Retweets,Followers,CleanTweet,CleanTweet2
0,2022-05-23 23:52:11+00:00,@elfrootloops I blame South Park for that one 😂,vee4vampy,0,839,"i, blame, south, park, for, that, one, 😂",i blame south park for that one
1,2022-05-23 23:50:20+00:00,@ClimateWarrior7 Avoid Regents Park Zoo. I hea...,oldboyinuk,0,344,"avoid, regents, park, zoo, i, hear, it's, a, p...",avoid regents park zoo i hear it's a pox-spot...
2,2022-05-23 23:47:37+00:00,next season i think we need to focus on breaki...,Kikirani19,0,376,"next, season, i, think, we, need, to, focus, o...",next season i think we need to focus on breaki...
3,2022-05-23 23:42:15+00:00,Gowran park \n\n16.50. 12.4.14\n17.20. 6.16\n1...,hollins48,0,736,"gowran, park, :",gowran park \n\n \n \n \n \n \n :\n \n
4,2022-05-23 23:41:46+00:00,@OffasTyke You aren’t alone! I watched two pe...,fascinatorfun,0,80198,"you, aren, ’, t, alone, i, watched, two, peopl...",you aren’t alone i watched two people one af...
...,...,...,...,...,...,...,...
9971,2022-05-19 16:52:11+00:00,Matlock to Darley Dale riverside walk \nDistan...,Coco_Travels,1,2998,"matlock, to, darley, dale, riverside, walk, di...",matlock to darley dale riverside walk \ndistan...
9972,2022-05-19 16:52:10+00:00,Our website is updated. Now we’re concentratin...,HALOApparelUK2,0,516,"our, website, is, updated, now, we, ’, re, con...",our website is updated now we’re concentrating...
9973,2022-05-19 16:52:00+00:00,"Campaigners have already raised £130,000\nhttp...",BristolLive,0,154009,"campaigners, have, already, raised, £",campaigners have already raised £\n
9974,2022-05-19 16:51:54+00:00,This afternoon I attended at Park Hill School ...,AkefAkbar,0,3112,"this, afternoon, i, attended, at, park, hill, ...",this afternoon i attended at park hill school ...


In [87]:
# Save DataFrame
tweets_df.to_csv('Park liverpool 2_3.csv', encoding='utf-8')